In [2]:
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 


import h5py
import cv2
import numpy as np
from models.gazeheadResnet import GazeHeadResNet
import torch
import torch.nn as nn
import time
from torch.utils.data import DataLoader

In [3]:
def preprocess_image(image):
     ycrcb = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
     ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
     image = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2RGB)
     image = np.transpose(image, [2, 0, 1])  # Colour image
     image = 2.0 * image / 255.0 - 1
     return image

def recover_image(x):
     x = (x + 1.0)*255.0/2.0
     x = np.clip(x, 0, 255) # Avoiding parts which have over or under flowed
     x = x.astype(np.uint0)
     
     x = np.transpose(x, [1, 2, 0]) # Channel, height and width to height width and channel

     x = x[:, :, ::-1] # Convertin  RGB to BGR since openCV uses that

     return x




torch.cuda.empty_cache()

gaze_head_resnet = GazeHeadResNet()

gaze_head_resnet = nn.DataParallel(gaze_head_resnet)
gaze_head_resnet.to("cuda")

DataParallel(
  (module): GazeHeadResNet(
    (resnet50): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): R

In [4]:

gaze_list = []
gaze_hat_list = []

#h5_path = "/bigpool/fachpraktikum2021/group5/dir_srinivas/MPIIGaze.h5"
h5_path = "/bigpool/fachpraktikum2021/group5/dir_srinivas/outputs_sted/GazeCapture.h5"
h5_path = "/bigpool/fachpraktikum2021/group5/dir_srinivas/outputs_sted/Redirected_samples1.h5"
f = h5py.File(h5_path, 'r')


#print("THe total number of images are: " + str(cnt))

In [5]:


vec = np.zeros((len(f.items()), 3, 128, 128), dtype=np.double)


with h5py.File(h5_path, 'r') as f:
    cnt = 0
    for person_id, group in f.items():
        #print(group["labels"][0, :2])
        eyes = preprocess_image(group["pixels"][0, :])

        if cnt < 10:
            cv2.imwrite("/bigpool/fachpraktikum2021/group5/dir_srinivas/outputs_sted/recovered_" + str(cnt) + ".png", recover_image(eyes))

        vec[cnt] = eyes.astype(np.double)

        gaze = group['labels'][0, :2]
        
        #print(vec)
        
        #Adding the entries for computing errors later on
        gaze_list.append(gaze)
        #gaze_hat_list.append(gaze_hat)

        cnt += 1

        #print("Finished processing : " + str(cnt) + " of " + str(len(f.items())))

print("Finished Computing the gaze")





Finished Computing the gaze


In [8]:
gaze_np = np.array(gaze_list)*(180/np.pi)
print(np.min(gaze_np))
print(np.max(gaze_np))

-36.349773
23.0211


In [6]:
#gaze_hat = gaze_hat.cpu().detach().numpy()
#del vec1
#del gaze_hat
#del head_hat
#del gaze_head_resnet

vec1 = torch.from_numpy(vec).float().to("cuda")
dataloader = DataLoader(vec1[:, ], batch_size=32)

for i, input in enumerate(dataloader):
    with torch.no_grad():
        gaze_hat, head_hat = gaze_head_resnet(input)
    gaze_hat_list.extend(gaze_hat.tolist())
    del gaze_hat
    del head_hat
    torch.cuda.empty_cache()
 

In [7]:
gaze = np.array(gaze_list) 
gaze_hat = np.array(gaze_hat_list)
#gaze_hat = gaze_hat.cpu().detach().numpy()
#vec1 = vec1.cpu().detach().numpy()



print(gaze_hat.shape)



print(np.sqrt(np.mean((gaze - gaze_hat)**2)))

#del vec1
torch.cuda.empty_cache()



(1364, 2)
0.4933277329006025
